# Stock Market Time Series Forecasting with Multiple Approaches

In this notebook we use historical stock market data for Apple Inc. (AAPL) loaded from a local CSV file (`HistoricalQuotes.csv`) to demonstrate several time series forecasting methods. We will predict the next day's closing price using different approaches, starting from the simplest to more advanced methods.

Below are the forecasting approaches we will cover:

1. **Naive Forecast:** The simplest method that assumes the next value is equal to the last observed value.
2. **Autoregressive (AR) Forecast:** Forecasting using the average of the previous *k* observations (here, *k* = 3).
3. **Exponential Smoothing Forecast:** A weighted average of past observations where more recent values are given higher weight.
4. **Linear Trend Forecast:** Fitting a linear model (a special case of linear regression) to capture the trend.
5. **Multiple Linear Regression:** Extending the linear trend model by incorporating additional features such as trading volume and moving averages.

The mathematical formulations for the methods are as follows:

### Naive Forecast

Assumes that the next value is the same as the last observed value:

$$ y_{t+1} = y_t $$

### Autoregressive (AR) Forecast

For a simple AR(3) model:

$$ y_{t+1} = \frac{y_t + y_{t-1} + y_{t-2}}{3} $$

### Exponential Smoothing Forecast

Forecast is computed as:

$$ F_{t+1} = \alpha y_t + (1-\alpha) F_t $$

with smoothing factor $\alpha$ (0 < $\alpha$ < 1).

### Linear Trend Forecast

A simple linear trend model assumes:

$$ y_t = a + b\,t $$

which is a special case of linear regression where time is the only predictor.

### Multiple Linear Regression

We extend the linear model to include additional predictors. For example, consider:

$$ y_t = \beta_0 + \beta_1\,t + \beta_2\,V_t + \beta_3\,MA_t + \epsilon_t $$

where $ V_t $ is the trading volume and $ MA_t $ is a moving average of the closing price. The coefficients $\beta$ are estimated via least squares:

$$ \beta = (X^T X)^{-1} X^T y $$

Let's now proceed with the implementation.

In [21]:
import numpy as np
import pandas as pd

# Set a display format for floats
pd.options.display.float_format = '{:.2f}'.format

## 1. Loading and Preparing the Data

We load the historical AAPL stock data from a local CSV file (`HistoricalQuotes.csv`). This dataset contains daily data such as Open, High, Low, Close, Adj Close, and Volume. In this example, we use the **Close** column for our analysis. 

The following steps are performed:

- Remove extra spaces from column names
- Rename the `Close/Last` column to `Close`
- Remove dollar signs from numeric columns and convert them to float
- Convert the `Volume` column to integers
- Parse the `Date` column as datetime and set it as the index

In [ ]:
dataset_path = "HistoricalQuotes.csv"

# Read the CSV data and parse the 'Date' column as dates
df = pd.read_csv(dataset_path, parse_dates=['Date'])

# Remove leading and trailing spaces from column names
df.columns = df.columns.str.strip()

# Rename 'Close/Last' to 'Close'
df.rename(columns={'Close/Last': 'Close'}, inplace=True)

# Remove dollar signs and convert the following columns to float
cols_to_convert = ["Close", "Open", "High", "Low"]
df[cols_to_convert] = df[cols_to_convert].replace({'\$': ''}, regex=True).astype(float)

# Convert 'Volume' to integer
df["Volume"] = df["Volume"].astype(int)

# Set 'Date' as the index
df.set_index('Date', inplace=True)

# Sort the index in ascending order
df = df.sort_index()

# Display the first few rows
df.head()

## 2. Visualizing the Data

Let's plot the daily closing price to get an initial look at the data.

In [ ]:
# Plot the Close price
ax = df['Close'].plot(title='AAPL Close Price in 2020', figsize=(10, 5))
ax.set_xlabel('Date')
ax.set_ylabel('Close Price')

## 3. Forecasting Approaches

We now implement several forecasting approaches. We start with the simplest method and proceed to more advanced approaches.

### 3.1 Naive Forecast

The **Naive Forecast** simply uses the previous day's closing price as the forecast for the next day. Mathematically:

$$ y_{t+1} = y_t $$

This method is easy to implement and serves as a benchmark for more sophisticated methods.

In [ ]:
# Create a copy of the data for the naive forecast
df_naive = df.copy()

# Shift the 'Close' column by one to get the naive forecast
df_naive['Naive Forecast'] = df_naive['Close'].shift(1)

df_naive.head(10)

### 3.2 Autoregressive (AR) Forecast

The **Autoregressive Forecast** (AR) uses the average of the previous 3 days' closing prices to predict the next day's closing price. The model is given by:

$$ y_{t+1} = \frac{y_t + y_{t-1} + y_{t-2}}{3} $$

This simple averaging can be seen as a basic AR(3) model.

In [ ]:
# Define the autoregressive prediction function with a lag of 3
def autoregression_predict(data, lag=3):
    """
    Predicts the next value as the average of the last 'lag' values.
    """
    return np.mean(data[-lag:])

# Extract the closing prices
close_prices = df['Close'].values

# Set the lag value
lag = 3
ar_predictions = []

# Loop over the data starting at index 'lag'
for i in range(lag, len(close_prices)):
    past_values = close_prices[i-lag:i]
    pred = autoregression_predict(past_values, lag=lag)
    ar_predictions.append(pred)

# Create a new DataFrame for AR forecast (available from the 'lag'-th observation)
df_ar = df.iloc[lag:].copy()
df_ar['AR Forecast'] = ar_predictions

df_ar.head(10)

### 3.3 Exponential Smoothing Forecast

The **Exponential Smoothing Forecast** computes a weighted average of past observations, with recent values receiving higher weight. The forecasting formula is:

$$ F_{t+1} = \alpha y_t + (1-\alpha) F_t $$

where $ \alpha $ is the smoothing factor. In this example, we use $ \alpha = 0.5 $.

In [ ]:
# Define exponential smoothing function
def exponential_smoothing(series, alpha):
    forecasts = [series[0]]  # initial forecast is the first observation
    for t in range(1, len(series)):
        forecast = alpha * series[t-1] + (1 - alpha) * forecasts[-1]
        forecasts.append(forecast)
    return np.array(forecasts)

# Set smoothing factor alpha
alpha = 0.5
exp_smooth_forecasts = exponential_smoothing(close_prices, alpha)

# Create a DataFrame for exponential smoothing forecast
df_exp = df.copy()
df_exp['Exp Smooth Forecast'] = exp_smooth_forecasts

df_exp.head(10)

### 3.4 Linear Trend Forecast

In the **Linear Trend Forecast**, we assume the data can be modeled by a linear relationship:

$$ y_t = a + b\,t $$

We use NumPy's `polyfit` function to fit a line to the data and then use the resulting linear model to generate forecasts.

In [ ]:
# Create a time index for the data
days = np.arange(len(close_prices))

# Fit a linear model (degree 1 polynomial)
coeff = np.polyfit(days, close_prices, deg=1)
linear_model = np.poly1d(coeff)

# Compute the linear trend forecast
linear_predictions = linear_model(days)

# Create a DataFrame for the linear trend forecast
df_linear = df.copy()
df_linear['Linear Trend Forecast'] = linear_predictions

df_linear.head(10)

## 4. Comparison of Forecasting Approaches

We now combine the forecasts from all approaches into a single DataFrame and visualize them along with the actual closing prices. Note that the AR forecast is only available from the index corresponding to the chosen lag (i.e. from the 4th observation onward).

In [ ]:
# Combine all forecasts into one DataFrame
df_combined = df.copy()

# Add Naive Forecast
df_combined['Naive Forecast'] = df_combined['Close'].shift(1)

# Add AR Forecast (fill beginning with NaN)
df_combined['AR Forecast'] = np.nan
df_combined.iloc[lag:, df_combined.columns.get_loc('AR Forecast')] = ar_predictions

# Add Exponential Smoothing Forecast
df_combined['Exp Smooth Forecast'] = exp_smooth_forecasts

# Add Linear Trend Forecast
df_combined['Linear Trend Forecast'] = linear_predictions

df_combined.head(15)

In [ ]:
# Plot all forecasting approaches along with the actual Close price
start_date = '2020-01-01'
end_date = '2020-02-15'

# ax = df_combined[["Close", "Naive Forecast", "AR Forecast", "Exp Smooth Forecast", "Linear Trend Forecast"]].plot(
#     title='Comparison of Forecasting Approaches',
#     figsize=(12, 6)
# )


ax = df_combined.loc[start_date:end_date, ["Close", "Naive Forecast", "AR Forecast", "Exp Smooth Forecast"]].plot(
    title='Comparison of Forecasting Approaches',
    figsize=(12, 6)
)

ax.set_xlabel('Date')
ax.set_ylabel('Price')
# ax.set_ylim(220, 360)
ax.figure.tight_layout()

## 5. Multiple Linear Regression Using Additional Features

In this section we extend the simple linear trend model by incorporating additional predictors. In addition to the time index $t$ (which captures the trend), we add:

- **Volume ($V_t$)**: The trading volume at time $t$.
- **Moving Average ($MA_t$)**: A simple moving average of the closing price computed over a 3-day window.

The multiple linear regression model takes the form:

$$ y_t = \beta_0 + \beta_1\,t + \beta_2\,V_t + \beta_3\,MA_t + \epsilon_t $$

We estimate the coefficients using the least squares solution:

$$ \beta = (X^TX)^{-1}X^Ty $$

Let's compute the additional features and fit the model.

In [ ]:
# Create a copy of the original dataframe for multiple linear regression
df_ml = df.copy()

# Create a time index as a new feature
df_ml['Time'] = np.arange(len(df_ml))

# Compute a 3-day moving average of the Close price
df_ml['Moving_Avg'] = df_ml['Close'].rolling(window=3).mean()

# Drop the first few rows with NaN values from the moving average calculation
df_ml = df_ml.dropna()

# Display the first few rows
df_ml.head(10)

In [ ]:
# Prepare the feature matrix X and target vector y
X = np.column_stack((
    np.ones(len(df_ml)),         # Intercept term
    df_ml['Time'].values,          # Time index
    df_ml['Volume'].values,        # Trading Volume
    df_ml['Moving_Avg'].values     # 3-day Moving Average
))

y = df_ml['Close'].values

# Solve for the coefficients using the least squares solution
beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
print("Estimated coefficients:", beta)

# Compute the predicted Close prices
y_pred = X @ beta

# Add the predictions to the dataframe
df_ml['MLR Forecast'] = y_pred

df_ml.head(10)

In [ ]:
# Plot the actual Close prices vs. the Multiple Linear Regression forecast

# ax = df_ml[['Close', 'MLR Forecast']].plot(title='Multiple Linear Regression: Actual vs Predicted Close', figsize=(10, 5))

ax = df_ml.loc[start_date:end_date, ['Close', 'MLR Forecast']].plot(title='Multiple Linear Regression: Actual vs Predicted Close', figsize=(10, 5))

ax.set_xlabel('Date')
ax.set_ylabel('Close Price')
ax.figure.tight_layout()

## Least Squares Solution in Multiple Linear Regression

In multiple linear regression, we aim to model the relationship between a target variable $ y $ and several predictor variables. The model is generally expressed as:

$$
y = X \beta + \epsilon
$$

where:

- **$ y $** is an $ n \times 1 $ vector of observed values (e.g., closing prices).
- **$ X $** is an $ n \times p $ matrix of predictors (features), where each row represents an observation and each column a predictor. Typically, the first column of $ X $ is a column of ones to account for the intercept.
- **$ \beta $** is a $ p \times 1 $ vector of coefficients we wish to estimate.
- **$ \epsilon $** is an $ n \times 1 $ vector of errors (or residuals), representing the difference between the observed and predicted values.

### The Objective

The goal of the least squares approach is to find the coefficient vector $ \beta $ that minimizes the sum of squared errors (SSE) between the observed values and the values predicted by our model. Mathematically, this objective is written as:

$$
\min_{\beta} \| y - X \beta \|^2
$$

This expression means we want to minimize the squared Euclidean norm of the error vector $ y - X \beta $.

### Deriving the Least Squares Solution

To find the optimal $ \beta $, we perform the following steps:

1. **Write the Error Function:**  
   The sum of squared errors is given by:
   $$
   S(\beta) = (y - X \beta)^T (y - X \beta)
   $$

2. **Differentiate with Respect to $ \beta $:**  
   To minimize $ S(\beta) $, we take its derivative with respect to $ \beta $ and set it to zero:
   $$
   \frac{\partial S(\beta)}{\partial \beta} = -2 X^T (y - X \beta) = 0
   $$

3. **Solve the Normal Equations:**  
   Rearranging the equation above yields the **normal equations**:
   $$
   X^T X \beta = X^T y
   $$

4. **Obtain the Least Squares Estimate:**  
   Provided that $ X^T X $ is invertible, we can solve for $ \beta $:
   $$
   \beta = (X^T X)^{-1} X^T y
   $$

This equation is the **least squares solution**. It gives us the coefficients that best fit the observed data in the sense of minimizing the squared errors.

## 6. Conclusion

In this notebook, we demonstrated multiple forecasting approaches for predicting AAPL's closing price:

- **Naive Forecast:** Assumes the next value is equal to the last observed value.
- **Autoregressive Forecast:** Uses the average of the previous 3 days' prices.
- **Exponential Smoothing Forecast:** Applies a weighted average with a smoothing factor $\alpha$ to give more importance to recent observations.
- **Linear Trend Forecast:** Fits a simple linear model (a special case of linear regression) using time as the predictor.
- **Multiple Linear Regression:** Extends the linear model by incorporating additional features such as trading volume and a moving average of the closing price.

Each method has its own assumptions and can serve as a baseline for more advanced forecasting techniques. Experimenting with these methods and tuning their parameters can provide insights into their performance on real-world data.

Feel free to extend this notebook further by incorporating other features or more sophisticated time series models.